In [1]:
# import tensorflow as tf
import numpy as np
import h2o
from h2o.estimators import H2ORandomForestEstimator
import os
import pandas as pd

# from .feature_engineering_filter import Find_correct_port

In [2]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [3]:
vessel_data = pd.read_csv("../Datasets/vessels.csv", delimiter="|")
port_data = pd.read_csv("../Datasets/ports.csv", delimiter="|")

In [4]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [5]:
train_data=train_data.merge(vessel_data[['vesselId', 'shippingLineId']], on='vesselId', how='left')

In [6]:
port_data_renamed=pd.DataFrame()
port_data_renamed[["portId","port_latitude","port_longitude"]]=port_data[["portId","latitude","longitude"]]
train_data=train_data.merge(port_data_renamed, on="portId", how="left")

In [7]:
print(train_data.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'shippingLineId', 'port_latitude',
       'port_longitude'],
      dtype='object')


In [8]:
train_data_preprocessed = train_data
train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  \
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

             shippingLineId  port_latitude  port_longitude  
0  61ec65aea8cafc0e93f0e9

In [9]:
train_latitude_radians = np.deg2rad(train_data_preprocessed["latitude"])
train_longitude_radians = np.deg2rad(train_data_preprocessed["longitude"])
train_cog_radians = np.deg2rad(train_data_preprocessed["cog"])
train_heading_radians = np.deg2rad(train_data_preprocessed["heading"])

port_latitude_radians = np.deg2rad(train_data_preprocessed["port_latitude"])
port_longitude_radians = np.deg2rad(train_data_preprocessed["port_longitude"])

train_hour = np.deg2rad(train_data_preprocessed["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_preprocessed["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_preprocessed["time"].dt.month * 360 / 12)


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)
train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

port_latitude_sin = np.sin(port_latitude_radians)
port_latitude_cos = np.cos(port_latitude_radians)
port_longitude_sin = np.sin(port_longitude_radians)
port_longitude_cos = np.cos(port_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["port_latitude_sin"] = train_latitude_sin
train_data_preprocessed["port_latitude_cos"] = train_latitude_cos
train_data_preprocessed["port_longitude_sin"] = train_longitude_sin
train_data_preprocessed["port_longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed["cog_sog_sin"] = train_data_preprocessed["cog_sin"]*train_data_preprocessed["sog"]
train_data_preprocessed["cog_sog_cos"] = train_data_preprocessed["cog_cos"]*train_data_preprocessed["sog"]

train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId","cog_sin","cog_cos","sog"], axis=1
)
print(train_data_preprocessed.head(10))

                 time  rot  navstat                  vesselId  \
0 2024-01-12 14:07:47 -6.0        0  61e9f38eb937134a3c4bfd8b   
1 2024-01-12 14:31:00  5.0        0  61e9f38eb937134a3c4bfd8b   
2 2024-01-12 14:57:23  5.0        0  61e9f38eb937134a3c4bfd8b   
3 2024-01-12 15:18:48  6.0        0  61e9f38eb937134a3c4bfd8b   
4 2024-01-12 15:39:47  7.0        0  61e9f38eb937134a3c4bfd8b   
5 2024-01-12 15:54:48  5.0        0  61e9f38eb937134a3c4bfd8b   
6 2024-01-12 16:14:59 -6.0        0  61e9f38eb937134a3c4bfd8b   
7 2024-01-12 16:35:24  2.0        0  61e9f38eb937134a3c4bfd8b   
8 2024-01-12 16:55:24 -1.0        0  61e9f38eb937134a3c4bfd8b   
9 2024-01-12 17:14:36  6.0        0  61e9f38eb937134a3c4bfd8b   

             shippingLineId  port_latitude  port_longitude  seconds_to_eta  \
0  61a8e672f9cba188601e84ab      13.263333       80.341111       -374867.0   
1  61a8e672f9cba188601e84ab      18.941944       72.885278        205140.0   
2  61a8e672f9cba188601e84ab      18.941944       7

In [10]:
print(train_data_preprocessed[["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]].describe())

       latitude_sin  latitude_cos  longitude_sin  longitude_cos
count  1.522065e+06  1.522065e+06   1.522065e+06   1.522065e+06
mean   5.674148e-01  7.325459e-01   4.657097e-02   5.090908e-01
std    3.533228e-01  1.287637e-01   5.297501e-01   6.767740e-01
min   -7.376648e-01  3.328656e-01  -1.000000e+00  -9.997826e-01
25%    5.666483e-01  6.245345e-01  -9.085523e-02  -9.307284e-02
50%    6.721562e-01  7.403239e-01   7.383488e-02   9.743727e-01
75%    7.809972e-01  8.197004e-01   3.145059e-01   9.969210e-01
max    9.429743e-01  1.000000e+00   9.923086e-01   1.000000e+00


In [11]:
def Last_known_location_training_data(
    data: pd.DataFrame, shift_lenghts
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data

    """
    all_test_data = pd.DataFrame()

    for shift_length in shift_lenghts:

        grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

        grouped_data["time_diff"] = (
            grouped_data["time"].diff(-shift_length).dt.total_seconds().abs()
        )

        original_time_and_id = grouped_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        shifted_data = grouped_data.shift(shift_length)
        shifted_data[
            [
                "last_latitude_sin",
                "last_latitude_cos",
                "last_longitude_sin",
                "last_longitude_cos",
            ]
        ] = shifted_data[
            ["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]
        ]

        shifted_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ] = original_time_and_id[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        # Drops all values with nan values
        result = shifted_data.dropna().reset_index(drop=True)

        all_test_data = pd.concat([all_test_data, result], ignore_index=True)

    # Uncomment the line below if you want to remove the "time" column after processing
    # data = data.drop("time", axis=1)

    return all_test_data

In [12]:
print(train_data_preprocessed.columns)

train_data_shifted_df = Last_known_location_training_data(
    train_data_preprocessed, [10,25]
)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'rot', 'navstat', 'vesselId', 'shippingLineId', 'port_latitude',
       'port_longitude', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'heading_sin', 'heading_cos', 'hour_sin', 'hour_cos', 'day_sin',
       'day_cos', 'month_sin', 'month_cos', 'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')


In [13]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [14]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time  rot  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16  0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16  0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  0.0   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34  0.0   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29  0.0   

   navstat            shippingLineId  port_latitude  port_longitude  \
0        5  61a8e672f9cba188601e84ac      31.140556      -81.496667   
1        5  61be24574ea00ae59d0fe388      14.808333      120.279444   
2        0  61ec6303a8cafc0e93f0e8f3      42.098889       11.780833   
3        1  61be24564ea00ae59d0fe37a     -43.606111      172.716111   
4        2  61ec94f1a8cafc0e93f0e92a      48.380556       -4.474167   

   seconds_to_eta  ...   day_cos  month_sin  month_cos  cog_sog_sin  \
0       -133396.0  ...  0.104528        0.

In [15]:
h2o.init(max_mem_size="26g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.1" 2024-10-15; OpenJDK Runtime Environment Homebrew (build 23.0.1); OpenJDK 64-Bit Server VM Homebrew (build 23.0.1, mixed mode, sharing)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmpu42na12e
  JVM stdout: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmpu42na12e/h2o_iverringheim_started_from_python.out
  JVM stderr: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmpu42na12e/h2o_iverringheim_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 11 days
H2O_cluster_name:,H2O_from_python_iverringheim_b74kmi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,25.98 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.9], seed=42)   
)               

In [17]:
features_lat = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "shippingLineId",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
]
features_long = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    "rot",
    "shippingLineId",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [22]:
params_lat_sin = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
}
params_lat_cos = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
}
params_long_sin = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
}
params_long_cos = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
}
# score_eval_metric_only if you want to predict only on the evaluation dataset, could help against overfitting

drf_lat_sin = H2ORandomForestEstimator(**params_lat_sin)
drf_lat_cos = H2ORandomForestEstimator(**params_lat_sin)
drf_long_sin = H2ORandomForestEstimator(**params_lat_sin)
drf_long_cos = H2ORandomForestEstimator(**params_lat_sin)


In [23]:
drf_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

drf Model Build progress: |

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,29.0,29.0,17612299.0000000,20.0,20.0,20.0,39126.0,59104.0,48391.414
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2024-11-09 16:36:21,0.016 sec,0.0,nan,nan,nan,nan,nan,nan
,2024-11-09 16:36:26,5.053 sec,1.0,0.0429655,0.0079911,0.0018460,0.0424346,0.0078955,0.0018007
,2024-11-09 16:36:34,13.102 sec,3.0,0.0374675,0.0073336,0.0014038,0.0307705,0.0066639,0.0009468
,2024-11-09 16:36:41,19.426 sec,5.0,0.0384860,0.0076403,0.0014812,0.0310879,0.0068409,0.0009665
,2024-11-09 16:36:47,25.468 sec,7.0,0.0424796,0.0093544,0.0018045,0.0326412,0.0082998,0.0010654
,2024-11-09 16:36:52,30.653 sec,9.0,0.0388835,0.0086071,0.0015119,0.0307912,0.0076708,0.0009481
,2024-11-09 16:36:58,36.705 sec,11.0,0.0356270,0.0079833,0.0012693,0.0286832,0.0071481,0.0008227
,2024-11-09 16:37:04,42.522 sec,13.0,0.0364535,0.0086201,0.0013289,0.0300275,0.0078381,0.0009017


In [24]:
drf_lat_cos.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,17.0,17.0,12525335.0000000,20.0,20.0,20.0,52877.0,64451.0,58711.176
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2024-11-09 16:37:58,0.013 sec,0.0,nan,nan,nan,nan,nan,nan
,2024-11-09 16:38:02,4.493 sec,1.0,0.0237678,0.0092771,0.0005649,0.0236421,0.0092417,0.0005589
,2024-11-09 16:38:09,11.431 sec,3.0,0.0165097,0.0056324,0.0002726,0.0129139,0.0050939,0.0001668
,2024-11-09 16:38:16,17.952 sec,5.0,0.0138305,0.0046674,0.0001913,0.0105461,0.0041308,0.0001112
,2024-11-09 16:38:20,22.197 sec,6.0,0.0149254,0.0052923,0.0002228,0.0117816,0.0047783,0.0001388
,2024-11-09 16:38:27,28.940 sec,8.0,0.0134712,0.0048358,0.0001815,0.0106914,0.0043292,0.0001143
,2024-11-09 16:38:33,35.424 sec,9.0,0.0143061,0.0052673,0.0002047,0.0116927,0.0047862,0.0001367
,2024-11-09 16:38:38,39.835 sec,10.0,0.0149386,0.0056127,0.0002232,0.0125718,0.0051655,0.0001581


In [26]:
performance_lat_sin = drf_lat_sin.model_performance()
performance_lat_cos = drf_lat_cos.model_performance()


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.0012203639920961204
RMSE: 0.034933708536256505
MAE: 0.009332908525591222
RMSLE: 0.04085693296772546
Mean Residual Deviance: 0.0012203639920961204
ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.00019288043746842708
RMSE: 0.013888140173127108
MAE: 0.005474132752039412
RMSLE: 0.0076931443038094835
Mean Residual Deviance: 0.00019288043746842708


In [ ]:
drf_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)
#MSE: 0.00023605270678827214

drf Model Build progress: |███████████

In [ ]:
drf_long_cos.train(
    x=features_long,  # .append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729705799805_7


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.00013470981756651316
RMSE: 0.011606455857259492
MAE: 0.003105913857507238
RMSLE: NaN
Mean Residual Deviance: 0.00013470981756651316

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.00019859001791834753
RMSE: 0.014092197057888011
MAE: 0.0033277104937216727
RMSLE: NaN
Mean Residual Deviance: 0.00019859001791834753

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-23 18:46:43  0.002 sec          0.0                0.67669504197939      0.6008440122333621     0.45791617983948835     0.6781214557626759    0.6021292653116523     0.45984870876569084
     2024-10-23 18:46:54  11.568 sec         1.0                0.6094456505815108    0.5410542783737177     0.37142400101272094     0.6107208730242907    0.5422067894169109     0.3729799847475518
     2024-10-23 18:47:00  17.248 sec         3.0                0.49465279528855227   0.4388954935998272     0.24468138788677837     0.49568336508525235   0.43983412602031796    0.24570199842223955
     2024-10-23 18:47:07  23.936 sec         5.0                0.401737776519609     0.3561275760683088     0.16139324108291933     0.4025712882408014    0.35687616655375765    0.1620636421158584
     2024-10-23 18:47:13  29.860 sec         7.0                0.3264032982190439    0.28900666457334895    0.1065391130882701      0.3270992983654775    0.28961426108268107    0.10699395099118768
     2024-10-23 18:47:19  35.773 sec         9.0                0.2655191936865257    0.234725152997959      0.07050044221594275     0.26611589694822535   0.23521442156799452    0.0708176706085585
     2024-10-23 18:47:25  42.142 sec         11.0               0.2162785331304777    0.19073235616245504    0.046776403893071146    0.21680248848059588   0.1911242721112135     0.04700331901137891
     2024-10-23 18:47:31  48.282 sec         13.0               0.17646514440846206   0.15507416770648902    0.031139947191099367    0.1769301357787799    0.1553840424191338     0.031304272946697485
     2024-10-23 18:47:37  54.448 sec         15.0               0.14428446313317586   0.12613661369214       0.02081800630162878     0.14474930113862539   0.12639587488490958    0.02095236018012046
     2024-10-23 18:47:43  1 min  0.627 sec   17.0               0.11836618974454191   0.10270277561573968    0.014010554874640898    0.11884592497527459   0.10292245642124782    0.014124353883228595
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-23 18:51:41  4 min 58.015 sec   91.0               0.015352640142369922  0.0039019507036091717  0.00023570355934110835  0.017589720297745526  0.004076521868302299   0.00030939826015292097
     2024-10-23 18:52:08  5 min 25.244 sec   99.0               0.014948540231901434  0.003808910916911812   0.00022345885506477577  0.017155691879265158  0.0039850336953968805  0.0002943177638562844
     2024-10-23 18:52:38  5 min 55.471 sec   109.0              0.014364143714066585  0.003685899404222701   0.0002063286246383586   0.016645474356927308  0.003868937590515668   0.0002770718165671246
     2024-10-23 18:53:10  6 min 27.275 sec   119.0              0.013868854729601877  0.003577016170822063   0.00019234513151080034  0.01621233882866695   0.003766999481484866   0.0002628399302955021
     2024-10-23 18:53:46  7 min  3

In [ ]:
performance_long_sin = drf_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = drf_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00036365483471347996
RMSE: 0.019069736094489614
MAE: 0.005900173867516693
RMSLE: NaN
Mean Residual Deviance: 0.00036365483471347996
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00019859001791834753
RMSE: 0.014092197057888011
MAE: 0.0033277104937216727
RMSLE: NaN
Mean Residual Deviance: 0.00019859001791834753


In [ ]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = drf_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"] = lat_predictions_sin
lat_predictions_cos = drf_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = drf_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"] = long_predictions_sin
long_predictions_cos = drf_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"] = long_predictions_cos

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [ ]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

AttributeError: 'DataFrame' object has no attribute 'as_data_frame'

In [ ]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = drf_lat_sin.predict(validation_data)
    lat_val_cos = drf_lat_cos.predict(validation_data)
    long_val_sin = drf_long_sin.predict(validation_data)
    long_val_cos = drf_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("drf_eval_predictions.csv")


# create_prediction_visualization_data(validation_data_shifted)

In [ ]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
print(predictions.columns)

Index(['ID', 'longitude_predicted', 'latitude_predicted'], dtype='object')


In [ ]:
predictions.to_csv("drf_prediction.csv", index=False)